Building a Chatbot with history

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_KEY")

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")
llm

In [ ]:
from langchain_core.messages import HumanMessage
response=llm.invoke([HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai")])
response.content

In [ ]:
from langchain_core.messages import AIMessage
llm.invoke(
    [
        HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ,
        AIMessage(content="Hi Sachin,\n\nIt's great to meet you!  It's impressive that you're already practicing Generative AI during your B.Tech in CSE at Thapar University.  That's a very forward-looking skillset to be developing.  What specific areas of Generative AI are you focusing on right now?  I'd be interested to hear more about your projects.") ,
        HumanMessage(content="What is my name?")
    ]
)


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} #storing all the chat with their id to uniquely define and prevent chat mixing
def get_session_history(session_id)->BaseChatMessageHistory:
    if session_id not in store:  #if new chat then store the chat going on
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(llm,get_session_history)

In [ ]:
#declaring session id chat1 for this chat for a user
config={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke (
    [ HumanMessage(content="Hii, My name is Sachin Goyal.I am currently pursuing my B.Tech(3rd year) from Thapar University in CSE,i am currently practising Generative Ai.what will you suggest me for a better learning?") ]
    ,config=config )
response

In [ ]:
#see this time it says i don't know about you name
#basically the session id is a new one so no previous chat available it will be stored for the first time

config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response

In [ ]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="ok tell me what is my name")],config=config
)
response
#see it remembers my name

In [ ]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke([HumanMessage(content="My name is John")],config=config)
response

In [ ]:
config={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],config=config
)
response
#this time remembers the name

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.


HOW IT WORKS

-->we create a prompt where system command is there to llm

-->we create a variable of MessagePlaceholder which will place all the previous msg+new one

-->we create a chain of prompt and llm

-->for chat history we have Runnable with msgHistory which gets session id and
    retrieve all the prev msg history and provide it to chain

-->the llm will generate the response based on the prev history

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")]})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="My name is Sachin Goyal")]},config=config
)
response

In [ ]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {'message':[HumanMessage(content="can you tell me what is my name,age and my fav hobby")]},config=config
)
response

What if we have multiple inputs to be taken in prompt
--> then there is a change in with_message_history
    we need to define a correct key to use to save chat to history

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
     [
         ("system","You are a helpful assistant ,communicate with the user according in {language}"),
         MessagesPlaceholder(variable_name="messages")
     ]
 )
chain=prompt|llm

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hii ,My name is Sachin Goyal")],"language":"Hindi"})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

whenever we reuse the older session for ex chat1 then it loads the old history and as we used english by default so now declaring any other language may not work for that and will print in english

In [ ]:
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

In [ ]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Sachin Goyal")],"language":"Hindi"},config=config
)
response

In [ ]:
config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="can you tell me my name ")],"language":"Hindi"},config=config
)
response

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy='last', # keep the msgs from last with tokens<=max_tokens
    token_counter=llm,
    allow_partial=False, #If a msg pushes total tokens over 45, it won’t partially include it instead, it will stop adding messages.
    include_system=True, # as system guides the model what should it do so it must be present anyhow even after trimming msgs
    start_on='human'  #look for the most recent HumanMessage and start including from there.
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

maybe the trimmer trimmed the ice cream part here so not working check next code section

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
       | prompt
       | llm
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="what ice cream do i like")],"language":"Hindi"}
)
response

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
       | prompt
       | llm
)
response=chain.invoke(
    {"messages":messages+[HumanMessage(content="what was my question regarding maths")],"language":"Hindi"}
)
response

Wrapping with Message history (summary only)


In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [ ]:
config={"configurable":{"session_id":"chat6"}}
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="what is my name")],"language":"Hindi"},config=config
)
response

In [ ]:
config={"configurable":{"session_id":"chat6"}}
response=with_message_history.invoke(
    {"messages":messages+[HumanMessage(content="My name is Sachin,can you tell me the meaning of this ")],"language":"Hindi"},config=config
)
response